In [1]:
import numpy as np
from astropy.cosmology import FlatLambdaCDM

In [2]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [3]:
def radluminosity(nu1, nu2, S, alpha, z):
    #calculate luminosity at nu1 based on flux at nu2
    DL = 3.086e24*cosmo.luminosity_distance(z).value
    L = ((4*np.pi*DL**2*(S*1e-23*1e-6)) / ((1+z)**(1+alpha))) * (nu1/nu2)**alpha * 1e-7
    return np.log10(L)

def fluxdensity(nu1, nu2, L, alpha, z):
    #calculate flux density at nu2 based on luminosity at nu1
    #L = 10.**L
    DL = 3.086e24*cosmo.luminosity_distance(z).value #Mpc --> cm
    S = L * (nu2/nu1)**alpha * (((1+z)**(1+alpha))/(4*np.pi*DL**2)) * 1e36
    return S #return flux density in microJy

def SFR_L14(L14):
    #Yun+2001 SFR relation based on 1.4GHz luminosity
    return 5.9e-22 * (L14)

In [4]:
np.log10(10.**23)

23.0

In [5]:
fluxdensity(1.4, 1.4, 10.**24.5, -0.7, 1.65)

235.11603996180102

Magliocchetti find the following relationship between redshift and the break between SF/AGN radio luminosity functions:
$$ \log{P_\mathrm{1.4GHz}/\mathrm{[W/Hz/Sr]}} \approx 21.7 + z $$

Evaluating at $z=1.7$, we find 
$$ P_\mathrm{1.4GHz} = 2.5\times10^{23} \mathrm{W/Hz/Sr} \rightarrow L_\mathrm{1.4GHz} = 3.2\times10^{24} \mathrm{W/Hz} $$

---

Now we ask, what is the corresponding 3GHz flux density for a source $N\times$ below that break.

5x below break: $ \frac{3.2\times10^{24} \mathrm{W/Hz}}{5} = 6.3\times10^{23} \mathrm{W/Hz} \rightarrow \dot{M} \sim 370\,M_\odot\,\mathrm{yr}^{-1} $

8x below break: $ \frac{3.2\times10^{24} \mathrm{W/Hz}}{8} = 3.9\times10^{23} \mathrm{W/Hz} \rightarrow \dot{M} \sim 230\,M_\odot\,\mathrm{yr}^{-1} $

10x below break: $ \frac{3.2\times10^{24} \mathrm{W/Hz}}{10} = 3.2\times10^{23} \mathrm{W/Hz} \rightarrow \dot{M} \sim 190\,M_\odot\,\mathrm{yr}^{-1} $

20x below break: $ \frac{3.2\times10^{24} \mathrm{W/Hz}}{20} = 1.6\times10^{23} \mathrm{W/Hz} \rightarrow \dot{M} \sim 95\,M_\odot\,\mathrm{yr}^{-1} $

In [6]:
np.log10(6.3e23), np.log10(3.9e23), np.log10(3.2e23), np.log10(1.6e23)

(23.799340549453582, 23.5910646070265, 23.505149978319906, 23.204119982655925)

In [7]:
SFR_L14(6.3e23), SFR_L14(3.9e23), SFR_L14(3.2e23), SFR_L14(1.6e23)

(371.70000000000005, 230.1, 188.8, 94.4)

These are *luminosities* at *1.4GHz*.  We want *flux densities* at *3GHz*.

In [8]:
print("3GHz flux 5x below RLF : %5.1f µJy" % fluxdensity(1.4, 3., 6.3e23, -0.7, 1.7))
print("3GHz flux 8x below RLF : %5.1f µJy" % fluxdensity(1.4, 3., 3.9e23, -0.7, 1.7))
print("3GHz flux 10x below RLF: %5.1f µJy" % fluxdensity(1.4, 3., 3.2e23, -0.7, 1.7))
print("3GHz flux 20x below RLF: %5.1f µJy" % fluxdensity(1.4, 3., 1.6e23, -0.7, 1.7))
#print("Flux limit")

3GHz flux 5x below RLF :  25.7 µJy
3GHz flux 8x below RLF :  15.9 µJy
3GHz flux 10x below RLF:  13.0 µJy
3GHz flux 20x below RLF:   6.5 µJy


VLA's confusion limit for C-config in S-band is ~2.4uJy, so we can forget about going 20x deeper for all sources that need it -- that would require ~29hr to reach!

In [9]:
3.2e24 / 10.**radluminosity(1.4, 3., 30, -0.7, 1.65)

4.651743169705583

In [10]:
SFR_L14(10.**radluminosity(1.4, 3., 30, -0.7, 1.65))

405.8693550184747

Before, we went <~5x below the knee of the RLF, reaching SFRs down to ~400Msol/yr.

So, we may ask to go down to 4.4uJy (2 hr per source) to go down to ~200Msol/yr.

Should be able to save some time if we observe for <1hour to start, then keep exposing for the full 2hr if a detection is not achieved.